# 深度学习稳分核心：Stratified K-Fold 交叉验证指南

这是 Kaggle 比赛中提分的“核武器”。很多时候，单模型调参收益微乎其微，但通过 K-Fold 融合，分数往往能显著提升。

---

## 1. 什么是 Stratified K-Fold？（通俗版）

**核心逻辑：** 不浪费每一张图片，让每一张图既做过训练集，也做过验证集。

想象你有一块披萨（数据集），你要把它切成 5 份（5-Fold）：

* **普通 KFold**：随便切 5 刀。可能第一块全是香肠，第二块全是芝士。这会导致模型在第一块上训练时没见过芝士，验证时却考了芝士，导致分数剧烈波动。
* **Stratified (分层) KFold**：它会先看清披萨上的配料比例（比如 70% 香肠，30% 芝士）。切的时候，它保证**每一小块**里，香肠和芝士的比例都严格保持 7:3。

**结论：** `StratifiedKFold` 保证了验证集（Validation Set）的分布和训练集、测试集高度一致，这样跑出来的分数才“稳”。

---

## 2. PyTorch 核心代码模板

这是 PyTorch 配合 Sklearn 实现 K-Fold 的标准写法。
**核心技巧：** 我们只切分索引 (Index)，不物理移动或复制图片文件。

```python
import numpy as np
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, Subset
import torch

# ==========================================
# 1. 准备数据
# ==========================================
# 假设 dataset 是你的全量数据集 (例如 ImageFolder 或自定义 Dataset)
# 关键点：必须提取出所有的 label 列表，StratifiedKFold 需要根据 label 来平衡切分
# 如果是 ImageFolder，可以通过 dataset.targets 获取
all_labels = dataset.targets  
# 如果是自定义 dataset，可能需要类似 [img[1] for img in dataset] 这样的列表来获取标签

# ==========================================
# 2. 初始化 K-Fold
# ==========================================
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# skf.split 需要 X 和 y。
# 在深度学习中 X (图片数据) 太大，我们通常只传一个占位符 (np.zeros) 和 y (标签)
X_placeholder = np.zeros(len(all_labels))

# ==========================================
# 3. 开始循环训练 (Training Loop)
# ==========================================
# enumerate 会返回当前是第几折 (fold_idx)，以及这一折的 训练集索引 和 验证集索引
for fold_idx, (train_index, val_index) in enumerate(skf.split(X_placeholder, all_labels)):
    
    print(f"\n====== 开始训练第 {fold_idx + 1} / {n_splits} 折 ======")
    
    # --- [核心步骤]：利用 Subset 创建这一折的数据集 ---
    #Subset 不会复制数据，只是创建了一个索引映射，非常节省内存
    train_subset = Subset(dataset, train_index)
    val_subset = Subset(dataset, val_index)
    
    # --- 创建 DataLoader ---
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=4)
    
    # --- 初始化模型 (非常重要！) ---
    # 每一折必须是一个全新的模型，不能接着上一折训练！
    # 否则这就不是交叉验证，而是继续训练了。
    model = MyCNN() 
    model = model.to(device)
    
    # --- 初始化优化器和损失函数 ---
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    
    # --- 训练代码 (伪代码) ---
    # for epoch in range(epochs):
    #     train_one_epoch(...)
    #     val_acc = validate(...)
    #     
    #     # 保存这一折最好的模型
    #     if val_acc > best_acc:
    #         torch.save(model.state_dict(), f"model_fold_{fold_idx}.pth")
    
    print(f"第 {fold_idx + 1} 折训练结束，模型已保存。")

In [3]:
import os  # [语法]: 导入标准库 os (Operating System)。 [作用]: 提供与操作系统交互的功能，比如路径拼接、创建文件夹、列出文件名。
import shutil  # [语法]: 导入标准库 shutil (Shell Utilities)。 [作用]: 提供高级文件操作，最主要就是用来复制(copy)和移动文件。
import pandas as pd  # [语法]: 导入 pandas 库并起别名 pd。 [作用]: 数据分析的神器，这里专门用来读取和处理 .csv 格式的标签文件。
from tqdm import tqdm  # [语法]: 从 tqdm 库导入 tqdm 模块。 [作用]: 显示进度条。在处理成千上万张图片时，能让你知道代码跑到哪了，不至于干等。

# 1. 定义路径 (根据你的实际存放位置修改)
# [语法]: 变量赋值 (字符串)。
# [作用]: 定义所有数据存放的根目录。所有的子文件夹都会在这个目录下。
DATA_ROOT = './kaggle_dog'

# [语法]: os.path.join 路径拼接。
# [作用]: 拼接出原始训练集图片的存放路径 (例如 ./kaggle_dog/train)。它会自动适配 Windows(\) 和 Linux(/) 的分隔符。
ORIGIN_TRAIN_DIR = os.path.join(DATA_ROOT, 'train')

# [语法]: os.path.join 路径拼接。
# [作用]: 拼接出原始测试集图片的存放路径。
ORIGIN_TEST_DIR = os.path.join(DATA_ROOT, 'test')

# [语法]: os.path.join 路径拼接。
# [作用]: 拼接出标签文件 labels.csv 的完整路径。
LABEL_CSV = os.path.join(DATA_ROOT, 'labels.csv')

# 目标路径: 我们把整理好的图片放到这里
# [语法]: os.path.join 路径拼接。
# [作用]: 定义一个新文件夹的路径，准备用来存放整理后(按品种分类)的训练图片。
TARGET_TRAIN_DIR = os.path.join(DATA_ROOT, 'train_ready')

# [语法]: os.path.join 路径拼接 (多层)。
# [作用]: 定义整理后的测试集路径。
# ★关键点★：为什么要加 'unknown'？因为 PyTorch 的 ImageFolder 读取数据时，强制要求格式为 root/类别名/图片。
# 测试集没有类别，所以我们需要人为制造一个伪类别文件夹(unknown)，否则代码会报错。
TARGET_TEST_DIR = os.path.join(DATA_ROOT, 'test_ready', 'unknown') 

# [语法]: 定义函数 organize_data。
# [作用]: 封装数据整理的所有逻辑，方便一键调用。
def organize_data():
    
    # [语法]: os.path.exists 判断路径是否存在。
    # [作用]: 这是一个“防呆设计”。如果代码以前跑过，文件夹已经有了，就直接跳过，避免重复复制浪费时间。
    if os.path.exists(TARGET_TRAIN_DIR):
        
        # [语法]: 打印提示信息。
        # [作用]: 告诉用户程序为什么没有干活。
        print("数据好像已经整理好了，跳过此步。")
        
        # [语法]: return 语句。
        # [作用]: 结束当前函数的执行，直接跳出。
        return

    # [语法]: 打印提示信息。
    # [作用]: 提示用户程序开始处理训练集了。
    print("开始整理训练集数据 (Copied)...")
    
    # [语法]: pd.read_csv 读取 CSV 文件。
    # [作用]: 将 labels.csv 读取为一个 DataFrame (表格对象)，方便后续查每一张图对应的品种。
    df = pd.read_csv(LABEL_CSV)
    
    # [语法]: for 循环 + tqdm 进度条 + df.iterrows() 迭代器。
    # [作用]: 
    # 1. df.iterrows(): 把表格一行一行地取出来。
    # 2. tqdm(...): 给这个循环加个进度条，显示 "处理到第几行了 / 总共多少行"。
    # 3. total=len(df): 告诉进度条总数，方便计算百分比。
    for idx, row in tqdm(df.iterrows()):
        
        # [语法]: 字典/Series 取值。
        # [作用]: 获取当前这一行里的 'id' 列的值 (即图片文件名，不含后缀)。
        img_id = row['id']
        
        # [语法]: 字典/Series 取值。
        # [作用]: 获取当前这一行里的 'breed' 列的值 (即狗的品种，也就是我们要创建的文件夹名)。
        breed = row['breed']
        
        # [语法]: os.path.join + f-string 格式化字符串。
        # [作用]: 拼凑出这张图片现在的完整路径。例如: ./kaggle_dog/train/001.jpg。
        src = os.path.join(ORIGIN_TRAIN_DIR, f"{img_id}.jpg")
        
        # [语法]: os.path.join 路径拼接。
        # [作用]: 拼凑出这张图片想去的地方。例如: ./kaggle_dog/train_ready/husky。
        dst_dir = os.path.join(TARGET_TRAIN_DIR, breed)
        
        # [语法]: if 判断 + not 取反。
        # [作用]: 检查目标品种文件夹是否存在。如果不存在(not exists)，才需要创建。
        if not os.path.exists(dst_dir):
            
            # [语法]: os.makedirs 创建目录。
            # [作用]: 创建对应的品种文件夹。makedirs 支持递归创建(如果上级目录也没有会一起创建)。
            os.makedirs(dst_dir)
            
        # [语法]: shutil.copy 文件复制。
        # [作用]: 核心动作！把图片从源路径(src) 复制到 目标文件夹下的同名文件。
        # f"{img_id}.jpg" 是为了保持文件名不变。
        shutil.copy(src, os.path.join(dst_dir, f"{img_id}.jpg"))

    # [语法]: 打印提示信息。
    # [作用]: 训练集搞定，提示开始搞测试集。
    print("开始整理测试集数据...")
    
    # [语法]: if 判断路径是否存在。
    # [作用]: 检查测试集的目标文件夹 ./kaggle_dog/test_ready/unknown 是否存在。
    if not os.path.exists(TARGET_TEST_DIR):
        
        # [语法]: os.makedirs 创建目录。
        # [作用]: 如果不存在，就创建这个目录。
        os.makedirs(TARGET_TEST_DIR)
        
    # [语法]: for 循环 + os.listdir 列出文件。
    # [作用]: 遍历原始测试集文件夹(ORIGIN_TEST_DIR)下的所有文件名。
    # tqdm 用来显示进度。
    for img_name in tqdm(os.listdir(ORIGIN_TEST_DIR)):
        
        # [语法]: os.path.join 路径拼接。
        # [作用]: 确定这张测试图现在的绝对路径。
        src = os.path.join(ORIGIN_TEST_DIR, img_name)
        
        # [语法]: os.path.join 路径拼接。
        # [作用]: 确定这张测试图要复制去的地方 (统一放到 unknown 文件夹下)。
        dst = os.path.join(TARGET_TEST_DIR, img_name)
        
        # [语法]: shutil.copy 文件复制。
        # [作用]: 执行复制操作。
        shutil.copy(src, dst)
        
    # [语法]: 打印提示信息。
    # [作用]: 告诉用户所有工作全部完成，可以进行下一步训练了。
    print("所有数据整理完毕！")

# [语法]: 函数调用。
# [作用]: 真正执行上面定义好的 organize_data 函数，开始干活。
organize_data()

数据好像已经整理好了，跳过此步。


In [4]:
import torchvision.transforms as transforms  
# [语法]: 导入 torchvision.transforms 模块，并简写为 transforms。
# [作用]: 这是一个图像处理工具箱，提供了裁剪、翻转、颜色调整、归一化等一系列功能。

# ===========================
# 1. 定义训练集预处理策略
# ===========================
# [语法]: 变量赋值。
# [作用]: 训练集的数据增强策略。我们要故意把图片变得“难认”一点，强迫模型学到真正的特征，而不是死记硬背。
train_transform = transforms.Compose([
    
    # [语法]: 随机缩放裁剪 (RandomResizedCrop)。
    # 参数 224: 最终裁剪出的图片大小必须是 224x224 (这是 ResNet 模型的标准入口尺寸)。
    # scale=(0.08, 1.0): 随机选取原图 8% 到 100% 的区域。
    # ratio=(3.0/4.0, 4.0/3.0): 随机选取长宽比在 3/4 到 4/3 之间的区域。
    # [作用]: 
    # 这是最强的增强手段！模拟物体可能出现在图片的任何位置、任何大小（比如狗有时候在角落，有时候占满全屏）。
    # 它强迫模型去识别“狗的局部”也能认出是狗。
    transforms.RandomResizedCrop(224, scale=(0.08, 1.0), ratio=(3.0/4.0, 4.0/3.0)),
    
    # [语法]: 随机水平翻转 (RandomHorizontalFlip)。默认概率 p=0.5。
    # [作用]: 
    # 模拟“照镜子”。因为一只狗头朝左是狗，头朝右还是狗。
    # 这能直接让训练数据量“翻倍”，防止模型只认识头朝左的狗。
    transforms.RandomHorizontalFlip(),
    
    # [语法]: 颜色抖动 (ColorJitter)。
    # brightness=0.4: 亮度随机变化 ±40%。
    # contrast=0.4: 对比度随机变化 ±40%。
    # saturation=0.4: 饱和度随机变化 ±40%。
    # [作用]: 
    # 模拟不同的光照环境（比如白天、阴天、黄昏）。
    # 防止模型根据背景颜色来作弊（比如误以为“草地上的都是狗，沙发上的都是猫”）。
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    
    # [语法]: 转为 Tensor (ToTensor)。
    # [作用]: 
    # 1. 格式转换：把 PIL 图片 (整数 0-255, HxWxC) 变成 PyTorch Tensor (浮点数 0.0-1.0, CxHxW)。
    # 2. 维度重排：PyTorch 卷积层要求“通道数”在前，这一步自动处理了。
    # ★注意★：这一步必须在 Normalize 之前。
    transforms.ToTensor(),
    
    # [语法]: 标准化 (Normalize)。
    # 第一个列表是 RGB 三个通道的均值 (Mean)，第二个列表是标准差 (Std)。
    # [作用]: 
    # 公式：result = (input - mean) / std。
    # 为什么要用这几个奇怪的数字？因为这是 ImageNet 数据集百万张图片的统计结果。
    # 既然我们要用 ImageNet 预训练好的 ResNet，就必须把我们的图片分布拉到和 ImageNet 一样，模型才能正常工作。
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ===========================
# 2. 定义测试/验证集预处理策略
# ===========================
# [语法]: 变量赋值。
# [作用]: 测试集不需要“花里胡哨”的随机变换，我们需要一个稳定、固定的输入，来公平地测试模型到底准不准。
val_test_transform = transforms.Compose([
    
    # [语法]: 调整大小 (Resize)。
    # 参数 256: 把图片短边缩放到 256 像素，长边按比例缩放。
    # [作用]: 
    # 先把图变到 256 大小，比需要的 224 稍微大一点点，为下一步“中心裁剪”留出余地。
    # (如果不先 Resize 直接 Crop，可能会切掉太多东西)。
    transforms.Resize(256),
    
    # [语法]: 中心裁剪 (CenterCrop)。
    # 参数 224: 从图片的正中心，切出一块 224x224 的区域。
    # [作用]: 
    # 这是测试时的标准操作。我们默认物体大概率在图片中间。
    # 不使用 RandomCrop，因为测试必须是可复现的（每次跑结果都得一样）。
    transforms.CenterCrop(224),
    
    # [语法]: 转为 Tensor。
    # [作用]: 同上，转成模型能读懂的 0-1 之间的浮点张量。
    transforms.ToTensor(),
    
    # [语法]: 标准化。
    # [作用]: 同上，使用和训练集完全一样的参数进行归一化。
    # ★关键点★：测试集必须用和训练集一样的均值/标准差，千万不能重新算。
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# [语法]: 从 sklearn 库导入 StratifiedKFold 类
# [作用]: StratifiedKFold (分层K折) 是一种高级的数据切分工具。
#        普通的 KFold 只是随机切，可能会导致某一折里全是猫没有狗。
#        Stratified 版本会保证切出来的每一折里，猫狗比例都和原始数据一致（例如都是 7:3）。
from sklearn.model_selection import StratifiedKFold

# [语法]: 导入 numpy 库并简写为 np
# [作用]: Python 数据科学的核心库。在这里主要是为了配合 sklearn 处理索引数组，或者生成占位符数据。
import numpy as np

# [语法]: 从 PyTorch 工具包导入 DataLoader 和 Subset
# [作用]: 
#        DataLoader: 把数据打包成 Batch (一批一批)，负责洗牌(shuffle)和并行加速(num_workers)。
#        Subset: 交叉验证的神器。它不复制数据，而是通过索引(Index)从全量数据集中“虚拟”地抠出一部分作为训练集或验证集。
from torch.utils.data import DataLoader, Subset

# [语法]: 从 torchvision 导入 ImageFolder
# [作用]: PyTorch 官方提供的图像加载器。它默认你的文件夹结构是 "root/class_A/xxx.jpg"，并自动读取标签。
from torchvision.datasets import ImageFolder

# [语法]: 导入操作系统标准库 os
# [作用]: 用来处理文件路径，比如 os.path.join 拼接路径，适配 Windows/Linux 系统分隔符。
import os

# ============================================================
# 1. 准备全量数据集 (技巧：读两次，由 Subset 决定用哪个)
# ============================================================

# [语法]: 实例化 ImageFolder 类，创建一个数据集对象
# [作用]: 【专门给训练折用】。
#        注意 transform=train_transform，这意味着从这里取出的图片，会被随机裁剪、翻转、变色（数据增强）。
#        虽然名字叫 full_train，但实际训练时，我们只会在每一折里用 Subset 取出它的 80%。
full_train_dataset = ImageFolder(os.path.join(DATA_ROOT, 'train_ready'), transform=train_transform)

# [语法]: 再次实例化 ImageFolder 类，读取同一个文件夹
# [作用]: 【专门给验证折用】。
#        注意 transform=val_test_transform，这意味着从这里取出的图片，只是调整大小和归一化，没有随机干扰。
#        这是交叉验证的核心技巧：同一张图，作为训练集时要“乱动”，作为验证集时要“老实”。
full_val_dataset = ImageFolder(os.path.join(DATA_ROOT, 'train_ready'), transform=val_test_transform)

# ============================================================
# 2. 获取标签用于分层
# ============================================================

# [语法]: 访问对象的 targets 属性 (List 类型)
# [作用]: ImageFolder 会自动把类别文件夹转为数字索引 (0, 1, 2...)。
#        targets 存储了每一张图片的类别，例如 [0, 0, 1, 1, 2, ...]。
#        StratifiedKFold 需要这个列表来确保切分时各类别的比例均匀。
all_labels = full_train_dataset.targets


# ============================================================
# 3. 准备测试集
# ============================================================

# [语法]: 实例化 ImageFolder 读取测试集文件夹
# [作用]: 读取 Kaggle 的测试数据。注意使用 val_test_transform（不做增强），保证测试结果的确定性。
test_ds = ImageFolder(os.path.join(DATA_ROOT, 'test_ready'), transform=val_test_transform)

# [语法]: 实例化 DataLoader
# [作用]: 创建测试集的迭代器。
#        batch_size=64: 一次预测 64 张图。
#        shuffle=False: ★千万不能打乱★。测试集的顺序必须和文件名一致，否则提交结果时 ID 对不上就全错了。
#        num_workers=4: 启用 4 个 CPU 进程来预读取数据，加速运行。
test_iter = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=4)

# [语法]: f-string 格式化打印 len() 长度
# [作用]: 打印日志，确认数据量是否正确读入，防止读了个空文件夹还在那傻跑。
print(f"全量数据: {len(full_train_dataset)} 张")

# [语法]: 普通打印
# [作用]: 提示用户当前进度。
print("准备进行 5-Fold 交叉验证...")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [9]:
from torch import nn
import torchvision
class get_net(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = torchvision.models.resnet34(pretrained=True)
        for param in self.features.parameters():
        # [语法]: 设置 requires_grad = False
        # [作用]: 
        # 告诉 PyTorch 的自动求导引擎：“在反向传播时，不要计算这些参数的梯度！”
        # 结果就是：优化器（SGD/Adam）更新参数时，这部分的权重完全不会变。
        # 既节省了显存，又加快了训练速度，还保留了大神的智慧。
            param.requires_grad = False
        self.output_new = nn.Sequential(
            nn.Linear(1000, 256),
            nn.ReLU(),
            nn.Linear(256, 120)
        )
    def forward(self,x):
        x = self.features(x)
        x = self.output_new(x)
        return x


In [15]:

import torch
def evaluate_loss(data_iter, net, devices,loss):
    
    # [语法]: 多变量初始化
    # [作用]: 
    # l_sum: 用来累加所有图片的“总扣分” (Total Loss)。
    # n: 用来累加一共看了多少张图片 (Total Samples)。
    net.eval()
    l_sum, n = 0.0, 0
    
    # [语法]: 遍历 DataLoader
    # [作用]: 一批一批地从数据集中取出图片 (features) 和标签 (labels)。
    with torch.no_grad():
        for features, labels in data_iter:
            
            # [语法]: Tensor.to(device)
            # [作用]: 把数据搬运到 GPU 上 (devices[0])，因为模型在 GPU 上，数据也必须过去才能计算。
            features, labels = features.to(devices), labels.to(devices)
            
            # [语法]: 模型前向传播 (Forward)
            # [作用]: 让模型看图猜答案，得到预测结果 outputs。
            outputs = net(features)
            
            # [语法]: 计算损失
            # [作用]: 对比预测值 outputs 和真实值 labels。
            # ★关键★: 因为上面设了 reduction='none'，这里返回的 l 是一个向量 (Vector)，
            # 比如 batch_size=64，l 的形状就是 [64]，代表这 64 张图各自的 loss。
            l = loss(outputs, labels)
            
            # [语法]: l.sum() 求和
            # [作用]: 把这一个 batch 里所有图片的 loss 加起来，累加到总 loss (l_sum) 里。
            l_sum += l.sum().item()
            
            # [语法]: labels.numel() 获取元素个数 (Number of Elements)
            # [作用]: 统计这个 batch 里一共有多少张图 (比如 64 张)，累加到总数 n 里。
            n += labels.shape[0]
        
        # [语法]: 除法运算
        # [作用]: 总 Loss / 总图片数 = 平均 Loss。
        # 这就是我们要的最终指标：平均每一张图猜错的程度。
        return l_sum / n

In [ ]:
import torch
from torch import nn
import time
# [语法]: 从 sklearn 库导入 StratifiedKFold 类
# [作用]: StratifiedKFold (分层K折) 是一种高级的数据切分工具。
#        普通的 KFold 只是随机切，可能会导致某一折里全是猫没有狗。
#        Stratified 版本会保证切出来的每一折里，猫狗比例都和原始数据一致（例如都是 7:3）。
from sklearn.model_selection import StratifiedKFold

# [语法]: 导入 numpy 库并简写为 np
# [作用]: Python 数据科学的核心库。在这里主要是为了配合 sklearn 处理索引数组，或者生成占位符数据。
import numpy as np

# [语法]: 从 PyTorch 工具包导入 DataLoader 和 Subset
# [作用]: 
#        DataLoader: 把数据打包成 Batch (一批一批)，负责洗牌(shuffle)和并行加速(num_workers)。
#        Subset: 交叉验证的神器。它不复制数据，而是通过索引(Index)从全量数据集中“虚拟”地抠出一部分作为训练集或验证集。
from torch.utils.data import DataLoader, Subset

# [语法]: 从 torchvision 导入 ImageFolder
# [作用]: PyTorch 官方提供的图像加载器。它默认你的文件夹结构是 "root/class_A/xxx.jpg"，并自动读取标签。
from torchvision.datasets import ImageFolder

# [语法]: 导入操作系统标准库 os
# [作用]: 用来处理文件路径，比如 os.path.join 拼接路径，适配 Windows/Linux 系统分隔符。
import os

# ===========================
# 1. 全局超参数配置
# ===========================
n_splits = 5  
# [语法]: 整数 (int)
# [作用]: 定义交叉验证的折数。表示将数据集分为 5 份，轮流做验证集。

num_epochs = 15  
# [语法]: 整数 (int)
# [作用]: 定义训练轮数。15 轮通常配合 CosineAnnealingLR 刚好能让学习率降到一个周期。

lr = 1e-4  
# [语法]: 浮点数 (float)
# [作用]: 初始学习率 (Learning Rate)。1e-4 (0.0001) 是微调预训练模型时的常用保守值。

wd = 1e-4  
# [语法]: 浮点数 (float)
# [作用]: 权重衰减 (Weight Decay)。对应 L2 正则化项，用于防止模型过拟合。

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# [语法]: torch.device 对象
# [作用]: 硬件选择器。如果有显卡就用 'cuda'，否则用 'cpu'。

# ===========================
# 2. 初始化 K-Fold 分割器
# ===========================
skf = StratifiedKFold(
    n_splits=n_splits,  
    # [参数]: n_splits
    # [作用]: 告诉分割器要切几刀 (这里是5)。
    
    shuffle=True,       
    # [参数]: shuffle (bool)
    # [作用]: 是否打乱数据。True 表示切分前先洗牌，防止数据原本是按类别排序的导致切分不均。
    
    random_state=42     
    # [参数]: random_state (int)
    # [作用]: 随机种子。固定为 42 保证每次运行代码，洗牌的结果都一模一样 (可复现)。
)

X_placeholder = np.zeros(len(all_labels)) 
# [语法]: numpy 数组
# [作用]: 占位符。skf.split 强制要求传入 X (特征) 和 y (标签)，但我们只用 y 来分层，所以 X 造个假的糊弄过去。

# ===========================
# 3. K-Fold 主循环
# ===========================
# enumerate(skf.split(...)): 遍历每一折的切分结果
for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_placeholder, all_labels)):
    # fold_idx: 当前是第几折 (0, 1, 2, 3, 4)
    # train_idx: 这一折的训练集索引数组
    # val_idx: 这一折的验证集索引数组

    print(f"\n====== 开始训练第 {fold_idx + 1} / {n_splits} 折 ======")
    
    # ---------------------------
    # 3.1 数据集构建 (Data Preparation)
    # ---------------------------
    
    # 训练集子集
    train_subset = Subset(
        dataset=full_train_dataset,  
        # [参数]: dataset
        # [作用]: 数据源。这里传入带有“数据增强”预处理的全量数据集。
        
        indices=train_idx            
        # [参数]: indices
        # [作用]: 索引列表。告诉 Subset 只取全量数据里的哪些图片当作这一折的训练集。
    )
    
    # 验证集子集
    val_subset = Subset(
        dataset=full_val_dataset,    
        # [参数]: dataset
        # [作用]: 数据源。这里传入“不带增强”(只做Resize/Norm) 的全量数据集。
        # ★关键点★：验证集严禁做随机增强，必须保持原图评估。
        
        indices=val_idx              
        # [参数]: indices
        # [作用]: 这一折分配给验证集的索引。
    )
    
    # 训练加载器
    train_iter = DataLoader(
        dataset=train_subset,    
        # [参数]: dataset
        # [作用]: 要加载哪个数据集对象。
        
        batch_size=64,           
        # [参数]: batch_size (int)
        # [作用]: 每次喂给模型多少张图。64 是显存允许下的常用值。
        
        shuffle=True,            
        # [参数]: shuffle (bool)
        # [作用]: 是否打乱顺序。训练集必须 True，让模型每次看到的样本顺序不一样，学得更鲁棒。
        
        num_workers=4            
        # [参数]: num_workers (int)
        # [作用]: 开几个 CPU 进程来读图。4 表示用 4 个核并行读取，加快速度。
    )
    
    # 验证加载器
    val_iter = DataLoader(
        dataset=val_subset,      
        # [参数]: dataset
        # [作用]: 验证数据集。
        
        batch_size=64,           
        # [参数]: batch_size
        # [作用]: 验证时也可以批量验证。
        
        shuffle=False,           
        # [参数]: shuffle
        # [作用]: 验证集不需要打乱，False 即可 (打乱也没坏处，但没必要)。
        
        num_workers=4            
        # [参数]: num_workers
        # [作用]: 同上，加速读取。
    )
    
    # ---------------------------
    # 3.2 模型初始化 (Model Init)
    # ---------------------------
    net = get_net()       # [作用]: 获取一个新的、未训练过的 ResNet 模型结构
    net = net.to(device)  # [作用]: 将模型的所有权重矩阵搬运到 GPU 显存中
    
    # ---------------------------
    # 3.3 优化器配置 (Optimizer)
    # ---------------------------
    trainer = torch.optim.SGD(
        params=[p for p in net.parameters() if p.requires_grad], 
        # [参数]: params (list)
        # [作用]: 告诉优化器要更新哪些参数。这里过滤掉了被冻结(requires_grad=False)的层，只更新最后的全连接层。
        
        lr=lr,                
        # [参数]: lr (float)
        # [作用]: 学习率。决定每次参数更新跨多大步。
        
        momentum=0.9,         
        # [参数]: momentum (float)
        # [作用]: 动量。让梯度下降具有“惯性”，遇到坑洼能冲过去，加速收敛。0.9 是标准值。
        
        weight_decay=wd       
        # [参数]: weight_decay (float)
        # [作用]: 权重衰减系数。防止模型参数变得太大太复杂(过拟合)。
    )
    
    # ---------------------------
    # 3.4 学习率调度器 (Scheduler)
    # ---------------------------
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer=trainer,    
        # [参数]: optimizer
        # [作用]: 绑定上面的优化器，调度器会修改这个 trainer 里的 lr 属性。
        
        T_max=num_epochs,     
        # [参数]: T_max (int)
        # [作用]: 周期长度。设置为 num_epochs 表示学习率会在整个训练过程中，从 lr 降到 0 完成半个余弦周期。
        
        eta_min=0             
        # [参数]: eta_min (float)
        # [作用]: 最小学习率。即曲线下降的终点值 (0)。
    )
    
    # ---------------------------
    # 3.5 损失函数 (Loss Function)
    # ---------------------------
    loss_fn = nn.CrossEntropyLoss(
        reduction='none'      
        # [参数]: reduction (str)
        # [作用]: 决定返回值格式。
        # 'mean': (默认) 返回一个 Batch 的平均 Loss (标量)。
        # 'none': 返回一个 Batch 里 64 个样本各自的 Loss (向量 [64])。
        # 这里用 none 是为了方便后面自己统计 sum，其实用 mean 也可以，写法不同而已。
    )
    
    # ---------------------------
    # 3.6 训练循环 (Training Loop)
    # ---------------------------
    for epoch in range(num_epochs):
        
        net.train()           
        # [作用]: 开启训练模式。启用 Dropout 和 BatchNorm。
        
        train_l_sum, train_n = 0.0, 0 
        # [作用]: 初始化累加器 (Loss 总和，样本总数)
        
        start_time = time.time()      
        # [作用]: 记录开始时间
        
        # Batch 循环
        for features, labels in train_iter:
            # features: 图片张量 [64, 3, 224, 224]
            # labels: 标签张量 [64]
            
            features = features.to(device) # [作用]: 图片搬到 GPU
            labels = labels.to(device)     # [作用]: 标签搬到 GPU
            
            trainer.zero_grad()            
            # [作用]: 梯度清零。清除上一步残留的梯度信息。
            
            output = net(features)         
            # [作用]: 前向传播。计算预测结果。
            
            l = loss_fn(output, labels).sum() 
            # [作用]: 计算 Loss。因为 reduction='none'，所以 l 是个向量，要 .sum() 变成标量才能反向传播。
            
            l.backward()                   
            # [作用]: 反向传播。计算梯度。
            
            trainer.step()                 
            # [作用]: 参数更新。根据梯度修改权重。
            
            train_l_sum += l.item()        
            # [作用]: 累加 Loss 数值 (用于打印)。
            
            train_n += labels.shape[0]     
            # [作用]: 累加样本数 (labels.shape[0] 就是 batch_size，例如 64)。
        
        # 更新学习率 (每个 Epoch 结束后)
        scheduler.step()                   
        # [作用]: 根据余弦曲线，调低 trainer 里的学习率。
        
        # 验证集评估 (假设外部有 evaluate_loss 函数)
        valid_loss = evaluate_loss(val_iter, net, device,loss_fn)
        
        # 打印日志
        print(f'Fold {fold_idx+1} Epoch {epoch+1}: Train Loss {train_l_sum/train_n:.4f}, Val Loss {valid_loss:.4f}')
    
    # ---------------------------
    # 3.7 模型保存 (Save Model)
    # ---------------------------
    save_path = f'D:\\深度学习\\CNN 视觉与 Kaggle 实战\\mode_foldmodel_fold_{fold_idx}.pth' 
    # [作用]: 定义文件名
    
    torch.save(net.state_dict(), save_path)   
    # [参数]: net.state_dict()
    # [作用]: 获取模型当前的参数字典 (不包含模型结构，只包含权重数值)。
    # [参数]: save_path
    # [作用]: 保存路径。
    
    print(f"--> 第 {fold_idx+1} 折模型已保存为: {save_path}")

print("\n所有 K-Fold 训练完成！")


====== 开始训练第 1 / 5 折 ======


d:\python3.12\Anaconda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\python3.12\Anaconda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 1 Epoch 1: Train Loss 2.6894, Val Loss 0.8616
Fold 1 Epoch 2: Train Loss 1.4541, Val Loss 0.7194
Fold 1 Epoch 3: Train Loss 1.2840, Val Loss 0.6358
Fold 1 Epoch 4: Train Loss 1.2221, Val Loss 0.5959
Fold 1 Epoch 5: Train Loss 1.1587, Val Loss 0.5718
Fold 1 Epoch 6: Train Loss 1.1347, Val Loss 0.5693
Fold 1 Epoch 7: Train Loss 1.0595, Val Loss 0.5470
Fold 1 Epoch 8: Train Loss 1.0184, Val Loss 0.5277
Fold 1 Epoch 9: Train Loss 0.9750, Val Loss 0.5273
Fold 1 Epoch 10: Train Loss 0.9799, Val Loss 0.5060
Fold 1 Epoch 11: Train Loss 0.9167, Val Loss 0.5029
Fold 1 Epoch 12: Train Loss 0.9021, Val Loss 0.4865


In [ ]:
import torch.nn.functional as F 
# [语法]: 导入 PyTorch 的函数式接口模块，并简写为 F。
# [作用]: 我们需要用到 F.softmax 函数，把模型输出的原始分数 (Logits) 变成概率值。

print("开始进行模型融合预测 (Ensemble Prediction)...")

# ===========================
# 1. 初始化累加器
# ===========================
# [语法]: torch.zeros(行数, 列数)。
# [作用]: 创建一个全是 0 的大表格 (Tensor)，用来存放所有模型的预测结果之和。
# len(test_ds): 测试集一共有多少张图 (比如 10357 张)。
# 120: 每一张图要预测 120 个类别的概率。
# ★关键★: 这里默认是在 CPU 上创建的。不要把它 .to(device) 到 GPU，
# 因为测试集很大，存 5 个模型的结果容易把显存撑爆 (OOM)。
final_preds = torch.zeros(len(test_ds), 120) 

# ===========================
# 2. 遍历 5 个模型 (K-Fold 循环)
# ===========================
# [语法]: range(n_splits) 生成序列 [0, 1, 2, 3, 4]。
# [作用]: 我们训练了 5 折，所以要加载 5 个不同的模型文件 (.pth) 分别进行预测。
for fold_idx in range(n_splits):
    print(f"正在加载并预测第 {fold_idx + 1} 个模型...")
    
    # --- A. 加载模型 ---
    # [语法]: 调用自定义函数 get_net()。
    # [作用]: 重新实例化一个“空壳”模型 (结构是 ResNet34，但参数是乱的)。
    net = get_net()
    
    # [语法]: f-string 字符串格式化。
    # [作用]: 构造文件名，例如 'model_fold_0.pth', 'model_fold_1.pth'。
    model_path = f'D:\\深度学习\\CNN 视觉与 Kaggle 实战\\mode_foldmodel_fold_{fold_idx}.pth' 
    
    # [语法]: net.load_state_dict(torch.load(...))。
    # [作用]: 1. torch.load 把硬盘上的权重文件读进内存。
    # 2. load_state_dict 把这些权重填进上面的“空壳”模型里。
    # 现在 net 就是一个训练好的大神模型了。
    net.load_state_dict(torch.load(model_path))
    
    # [语法]: .to(device)。
    # [作用]: 把模型搬运到 GPU 上，加速推理。
    net = net.to(device)
    
    # [语法]: .eval()。
    # [作用]: ★极重要★ 开启评估模式。
    # 1. 关闭 Dropout (不再随机丢弃神经元)。
    # 2. 锁定 BatchNorm (使用训练好的全局均值/方差，而不是当前 Batch 的)。
    # 如果不写这句，预测结果会很不准，且不稳定。
    net.eval() 
    
    # [语法]: 初始化空列表。
    # [作用]: 临时存放当前这个模型 (fold_idx) 对测试集的预测结果。
    fold_preds = []
    
    # --- B. 开始推理 (Inference) ---
    # [语法]: 上下文管理器 no_grad。
    # [作用]: 告诉 PyTorch 不要计算梯度。
    # 预测时不需要反向传播，关掉它可以省一半显存，并加速计算。
    with torch.no_grad():
        # [语法]: 遍历测试集 DataLoader。
        # [作用]: 一批一批地取出测试图片。这里 _ 是占位符，因为测试集没有标签 (label)。
        # ★注意★: 这里的 test_iter 必须设置 shuffle=False (不能打乱顺序)，
        # 否则预测结果和文件名对应不上！
        for data, _ in test_iter:
            # [语法]: .to(device)。
            # [作用]: 把图片数据搬到 GPU 上。
            data = data.to(device)
            
            # [语法]: 前向传播。
            # [作用]: 让模型看图，输出预测值 (Logits)。
            # output 的形状是 [batch_size, 120]，数值范围是 (-inf, +inf)。
            output = net(data)
            
            # [语法]: Softmax 归一化。
            # [作用]: ★核心步骤★ 把 Logits 变成概率 (Probability)。
            # Logits: [2.5, -1.0, 0.5] -> 没法直接求平均。
            # Probs:  [0.8,  0.05, 0.15] -> 可以求平均。
            # dim=1 表示在类别维度上进行归一化 (让120个数加起来等于1)。
            probs = F.softmax(output, dim=1)
            
            # [语法]: .extend(iterable) 和 .cpu()。
            # [作用]: 
            # 1. .cpu(): 把预测结果从 GPU 拿回 CPU (为了省显存)。
            # 2. .extend(): 把这一批次的概率条目加入到 fold_preds 列表中。
            fold_preds.extend(probs.cpu())
            
    # --- C. 累加结果 ---
    # [语法]: torch.stack(list)。
    # [作用]: fold_preds 原本是一个包含 N 个 Tensor 的列表 (List of Tensors)。
    # stack 把它堆叠成一个大的 Tensor，形状变成 [Total_Images, 120]。
    # [逻辑]: 把当前模型 (Model A) 的预测概率，加到总账本 final_preds 上。
    # 相当于：Total = Model_A + Model_B + ...
    final_preds += torch.stack(fold_preds)

# ===========================
# 3. 取平均值 (Model Averaging)
# ===========================
# [语法]: Tensor 除法。
# [作用]: 总概率 / 模型数量 = 平均概率。
# 比如 5 个模型对同一张图预测是狗的概率分别为: 0.8, 0.9, 0.7, 0.8, 0.8
# 平均后: 4.0 / 5 = 0.8。这就是集成学习的威力，消除了个别模型的偏差。
final_preds = final_preds / n_splits
print("预测完成，开始生成 CSV...")

# ===========================
# 4. 生成提交文件 (Submission CSV)
# ===========================
# [语法]: os.listdir 和 sorted。
# [作用]: 获取测试集所有图片的文件名。
# ★致命细节★: sorted() 非常重要！
# 因为 test_iter (ImageFolder) 读取图片时是按文件名排序读取的。
# 所以我们这里生成 ID 列表时，也必须按文件名排序，才能和 final_preds 一一对应。
ids = sorted(os.listdir(os.path.join(DATA_ROOT, 'test_ready', 'unknown'))) 

# [语法]: 打开文件写入流。
with open('submission.csv', 'w') as f:
    # [语法]: f.write 写入字符串。
    # [作用]: 写入 CSV 的第一行 (表头)。
    # 格式: id,breed1,breed2,breed3...
    # .join(): 用逗号把所有类别名连起来。
    f.write('id,' + ','.join(full_train_dataset.classes) + '\n')
    
    # [语法]: zip 打包遍历。
    # [作用]: 同时遍历“图片文件名(ids)”和“预测概率(final_preds)”。
    # 就像拉拉链一样，把图片 ID 和它的预测结果配对。
    for i, output in zip(ids, final_preds):
        
        # [语法]: 字符串分割。
        # [作用]: 去掉文件名后缀。例如 '00a3edd22.jpg' -> split('.') -> ['00a3edd22', 'jpg'] -> [0] -> '00a3edd22'。
        # Kaggle 只要 ID，不要 .jpg。
        image_id = i.split('.')[0]
        
        # [语法]: 列表推导式 + 类型转换。
        # [作用]: output 是一个 Tensor (120个浮点数)。
        # num.item(): 把 Tensor 里的数取出来变成 Python float。
        # str(): 把 float 变成字符串，方便写入文件。
        str_probs = [str(num.item()) for num in output]
        
        # [语法]: 字符串拼接。
        # [作用]: 用逗号把 120 个概率值连成一个长字符串。
        csv_probs = ','.join(str_probs)
        
        # [语法]: 最终写入。
        # [作用]: 写入一行数据：ID + 逗号 + 120个概率 + 换行符。
        f.write(image_id + ',' + csv_probs + '\n')

print("恭喜！融合了 5 个模型的 submission_kfold.csv 生成完毕！")

64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
1280
1344
1408
1472
1536
1600
1664
1728
1792
1856
1920
1984
2048
2112
2176
2240
2304
2368
2432
2496
2560
2624
2688
2752
2816
2880
2944
3008
3072
3136
3200
3264
3328
3392
3456
3520
3584
3648
3712
3776
3840
3904
3968
4032
4096
4160
4224
4288
4352
4416
4480
4544
4608
4672
4736
4800
4864
4928
4992
5056
5120
5184
5248
5312
5376
5440
5504
5568
5632
5696
5760
5824
5888
5952
6016
6080
6144
6208
6272
6336
6400
6464
6528
6592
6656
6720
6784
6848
6912
6976
7040
7104
7168
7232
7296
7360
7424
7488
7552
7616
7680
7744
7808
7872
7936
8000
8064
8128
8192
8256
8320
8384
8448
8512
8576
8640
8704
8768
8832
8896
8960
9024
9088
9152
9216
9280
9344
9408
9472
9536
9600
9664
9728
9792
9856
9920
9984
10048
10112
10176
10240
10304
10357
提交文件 submission.csv 生成完毕！


In [ ]:
a = 'sdhsihd/shdskhd/sdhkahjd'
print(a.split('/'))

['sdhsihd', 'shdskhd', 'sdhkahjd']


In [ ]:
import torch
import sklearn